In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

2023-02-08 17:39:20.227483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
import autoencoder.aecExtraFeatures as Z_calculations

def addZToPrediction(model, data_point):
    reconstruction = model.decoder(model.encoder(data_point))

    Z_features = Z_calculations.getZVector(data_point, reconstruction)

    print(Z_features)
    for feature in Z_features:
        print(type(feature))

    Z_features_tensor = tf.convert_to_tensor(Z_features)

    data_point = tf.concat(data_point, Z_features_tensor)

    return data_point

In [3]:
def isAnomaly(data_point, model_1, model_2, threshold):

    # need autoencoder to return boolean isAnomaly
    isAnomaly = tf.math.less(tf.keras.losses.mae(model_1(data), data), threshold)

    # if the autoencoder doesn't find anything out of the ordinary, return False
    if not isAnomaly:
        return False

    data_point = addZToPrediction(model_1, data_point)

    # if the autoencoder sees something weird, run it through the isolation forest to make sure
    return model_2.predict(data_point)

In [4]:
train_data = pd.read_csv('eda_simple_classification/network_data_mod_train.csv')
test_data = pd.read_csv('eda_simple_classification/network_data_mod_test.csv')

frames = [train_data, test_data]

dataframe  = pd.concat(frames)
raw_data = dataframe.values

In [5]:
# The last element contains the labels
labels = raw_data[:, -1]

data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=34
)

In [6]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

2023-02-08 17:39:33.641022: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [8]:
from autoencoder.autoencoder import AnomalyDetector
autoencoder = AnomalyDetector()

In [9]:
autoencoder.compile(optimizer='adam', loss='mae')

In [10]:
history = autoencoder.fit(normal_train_data, normal_train_data,
          epochs=5,
          validation_data=(normal_test_data, normal_test_data),
          shuffle=True)

Epoch 1/5
Tensor("anomaly_detector/sequential/dense_2/Relu:0", shape=(None, 8), dtype=float32)
Tensor("anomaly_detector/sequential_1/dense_5/Sigmoid:0", shape=(None, 47), dtype=float32)
Tensor("anomaly_detector/sequential/dense_2/Relu:0", shape=(None, 8), dtype=float32)
Tensor("anomaly_detector/sequential_1/dense_5/Sigmoid:0", shape=(None, 47), dtype=float32)
1905/1927 [============================>.] - ETA: 0s - loss: 0.0144Tensor("anomaly_detector/sequential/dense_2/Relu:0", shape=(None, 8), dtype=float32)
Tensor("anomaly_detector/sequential_1/dense_5/Sigmoid:0", shape=(None, 47), dtype=float32)
1927/1927 [==============================] - 8s 3ms/step - loss: 0.0142 - val_loss: 4.1309e-04
Epoch 2/5
1927/1927 [==============================] - 4s 2ms/step - loss: 3.5210e-04 - val_loss: 3.5334e-04
Epoch 3/5
1927/1927 [==============================] - 4s 2ms/step - loss: 3.4082e-04 - val_loss: 3.1575e-04
Epoch 4/5
1927/1927 [==============================] - 4s 2ms/step - loss: 3.2043e

In [11]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Tensor("anomaly_detector/sequential/dense_2/Relu:0", shape=(None, 8), dtype=float32)
Tensor("anomaly_detector/sequential_1/dense_5/Sigmoid:0", shape=(None, 47), dtype=float32)
1927/1927 [==============================] - 3s 1ms/step
Threshold:  0.00045557792


In [15]:
autoencoder.encoder(normal_train_data[20000:20001])

[0.003314836649224162, 0.999994695186615]
<class 'float'>
<class 'float'>


ValueError: concat_dim: Tensor conversion requested dtype int32 for Tensor with dtype float32: <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.00331484, 0.9999947 ], dtype=float32)>

In [13]:
normal_train_data_with_Z = []
for i in range(1, len(normal_train_data)):
    normal_train_data_with_Z.append(addZToPrediction(autoencoder, normal_train_data[i-1:i]))

[0.003314836649224162, 0.999994695186615]
<class 'float'>
<class 'float'>


ValueError: concat_dim: Tensor conversion requested dtype int32 for Tensor with dtype float32: <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.00331484, 0.9999947 ], dtype=float32)>

In [ ]:
data_point = normal_train_data[:1]
data_point